In [11]:
!pip install librosa
!pip install soundfile
!pip install sounddevice

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
#for removing future warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
import os
import random
from tensorflow.python.platform import gfile
import librosa
import librosa.display as ld
from array import array
import soundfile
import sounddevice as sd

Import Keras seperti ini

In [2]:
# from tf.keras.models import Sequential  # Tidak bisa!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D, MaxPooling1D
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import Embedding, LSTM

In [3]:
print(tf.__version__)
print(tf.keras.__version__)

1.6.0
2.1.3-tf


# KONSTANTA & VARIABEL

In [4]:
DEBUG = False
PATH_TRAIN = os.getcwd()+'/word/train'
PATH_TEST = os.getcwd()+'/word/test'
LOGDIR = 'log/'
TEST_LOGDIR = 'log_test/'
LABEL_TO_INDEX_MAP = {}
WIDTH = 1025
HEIGHT = 1
NUM_LABELS = 0
BATCHRUN = 500
EPOCHS = 5
msframe = 320 #jumlah frame yang mau dianalisa
sequence = 5 #jumlah deret yang di ambil
DURATION = 80
gain=10
high=2000
low=100

In [5]:
def init(path):
    labels = os.listdir(path)
    index = 0
    for label in labels:
        LABEL_TO_INDEX_MAP[label] = index
        index += 1
        
    global NUM_LABELS
    NUM_LABELS = len(LABEL_TO_INDEX_MAP)
    print(LABEL_TO_INDEX_MAP)

def one_hot_encoding(label):
    encoding = [0] * len(LABEL_TO_INDEX_MAP)
    encoding[LABEL_TO_INDEX_MAP[label]] = 1
    return encoding

def get_batch(batch_size, path):
    X = []
    Y = []
    random.seed(5896)
    path = os.path.join(path,'*', '*.wav')
    waves = gfile.Glob(path)
    while True:
        random.shuffle(waves)
        for wave_path in waves:
            _,label = os.path.split(os.path.dirname(wave_path))
            X.append(get_spec(wave_path))
            Y.append(one_hot_encoding(label))

            if (len(X) == batch_size):
                yield X, Y
                X = []
                Y = []

def zerolistmaker(n):
    """Membuat list berisi 0, mirip seperti np.zeros"""
    listofzeros = [0] * n
    return listofzeros

def get_spec(wave_path, PAD_WIDTH=WIDTH):
    mg=[]
    wv, sr = soundfile.read(wave_path)
    delta_f = (high - low) / 79       #(80 - 1)
    fftsize = np.ceil(sr / delta_f).astype(int)
    low_bin = int(np.floor(low / delta_f))

    
    #print(wave_path)
    blocksize = int(sr * DURATION / 1000)
    #print('block:',blocksize)
    for i in range(0,len(wv),blocksize):
        magnitude = np.abs(np.fft.rfft(wv[i:i+blocksize], n=fftsize))
        magnitude *= gain / fftsize
        mg.append(magnitude.tolist())
    for i in range(0,75-len(mg)):
        mg.append(zerolistmaker(334))
    mg.reverse()
    return np.array(mg)

### Untuk memeriksa

# Keras Model

In [6]:
import glob
fil = glob.glob("./word/train/*/*.wav")

In [7]:
init(PATH_TRAIN)

{'bangun': 0, 'cipi': 1}


In [8]:
train_x = []
train_y = []
for i in fil:
    a = np.array(get_spec(i))
    #b = np.zeros((75,334))
    #masukkan matrix a ke dalam b(zeros)
    #b[:a.shape[0],:a.shape[1]] = a
    train_x.append(a)
    train_y.append(one_hot_encoding(i.split("\\")[-2]))

train_x=np.array(train_x)
train_y=np.array(train_y)
print(train_x.shape)
print(train_y.shape)

(88, 75, 334)
(88, 2)


In [16]:
model = Sequential()
#model3.add(Embedding(20000,128))

model.add(LSTM(500,dropout=0.2,recurrent_dropout=0.2,input_shape=(75,334),return_sequences=True,activation='sigmoid'))
#model.add(LSTM(1334,dropout=0.2,recurrent_dropout=0.2,input_shape=(75,334),return_sequences=True,activation='sigmoid'))
#model.add(LSTM(1334,dropout=0.2,recurrent_dropout=0.2,input_shape=(75,334),return_sequences=True,activation='sigmoid'))
#model.add(LSTM(1334,dropout=0.2,recurrent_dropout=0.2,input_shape=(75,334),return_sequences=True,activation='sigmoid'))
model.add(LSTM(500,dropout=0.2,recurrent_dropout=0.2,input_shape=(75,334),return_sequences=False,activation='sigmoid'))
#model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2,input_shape=(None,334),return_sequences=True))
#model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2,input_shape=(None,334),return_sequences=True))
#model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2,input_shape=(None,334),return_sequences=False))
model.add(Dense(128,activation="sigmoid"))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, None, 500)         1670000   
_________________________________________________________________
lstm_9 (LSTM)                (None, 500)               2002000   
_________________________________________________________________
dense_7 (Dense)              (None, 128)               64128     
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 258       
Total params: 3,736,386
Trainable params: 3,736,386
Non-trainable params: 0
_________________________________________________________________


In [18]:
from tensorflow.python.keras.optimizers import Adam
optimizer = Adam(lr=1e-4)

# Training

init(PATH_TRAIN)
batch = get_batch(BATCHRUN,PATH_TRAIN)
X, Y = next(batch)

x_train=[]
y_train=[]

for i in range(len(X)):
    ds=[]
    for ll in range(len(X[i])-sequence):
        ds = X[i][ll:ll+sequence]
        x_train.append(np.array(ds))
        y_train.append(Y[i])

print(np.array(x_train).shape)
print(np.array(y_train).shape)

init(PATH_TRAIN)
batch = get_batch(BATCHRUN,PATH_TRAIN)
X, Y = next(batch)


In [19]:
#MODEL TRAINING
model.fit(x=train_x, y=train_y, epochs=150, verbose=1)
print('...Training done')

Epoch 1/150
88/88 [==============================]88/88 [==============================] - 4s 50ms/step - loss: 0.6205 - acc: 0.7500

Epoch 2/150
88/88 [==============================]88/88 [==============================] - 3s 37ms/step - loss: 0.5784 - acc: 0.7500

Epoch 3/150
88/88 [==============================]88/88 [==============================] - 3s 37ms/step - loss: 0.5725 - acc: 0.7500

Epoch 4/150
88/88 [==============================]88/88 [==============================] - 3s 38ms/step - loss: 0.5650 - acc: 0.7500

Epoch 5/150
88/88 [==============================]88/88 [==============================] - 3s 38ms/step - loss: 0.5758 - acc: 0.7500

Epoch 6/150
88/88 [==============================]88/88 [==============================] - 3s 38ms/step - loss: 0.5711 - acc: 0.7500

Epoch 7/150
88/88 [==============================]88/88 [==============================] - 3s 37ms/step - loss: 0.5757 - acc: 0.7500

Epoch 8/150
88/88 [==============================]88/88 [=====

88/88 [==============================]88/88 [==============================] - 3s 37ms/step - loss: 0.5687 - acc: 0.7500

Epoch 62/150
88/88 [==============================]88/88 [==============================] - 4s 40ms/step - loss: 0.5625 - acc: 0.7500

Epoch 63/150
88/88 [==============================]88/88 [==============================] - 3s 38ms/step - loss: 0.5651 - acc: 0.7500

Epoch 64/150
88/88 [==============================]88/88 [==============================] - 3s 38ms/step - loss: 0.5619 - acc: 0.7500

Epoch 65/150
88/88 [==============================]88/88 [==============================] - 3s 38ms/step - loss: 0.5625 - acc: 0.7500

Epoch 66/150
88/88 [==============================]88/88 [==============================] - 3s 38ms/step - loss: 0.5772 - acc: 0.7500

Epoch 67/150
88/88 [==============================]88/88 [==============================] - 3s 38ms/step - loss: 0.5647 - acc: 0.7500

Epoch 68/150
88/88 [==============================]88/88 [==========

88/88 [==============================]88/88 [==============================] - 3s 37ms/step - loss: 0.5530 - acc: 0.7500

Epoch 122/150
88/88 [==============================]88/88 [==============================] - 4s 40ms/step - loss: 0.5573 - acc: 0.7500

Epoch 123/150
88/88 [==============================]88/88 [==============================] - 3s 38ms/step - loss: 0.5618 - acc: 0.7500

Epoch 124/150
88/88 [==============================]88/88 [==============================] - 3s 38ms/step - loss: 0.5570 - acc: 0.7500

Epoch 125/150
88/88 [==============================]88/88 [==============================] - 3s 39ms/step - loss: 0.5491 - acc: 0.7500

Epoch 126/150
88/88 [==============================]88/88 [==============================] - 3s 38ms/step - loss: 0.5542 - acc: 0.7500

Epoch 127/150
88/88 [==============================]88/88 [==============================] - 3s 38ms/step - loss: 0.5468 - acc: 0.7500

Epoch 128/150
88/88 [==============================]88/88 [===

# TEST PREDIKSI

In [477]:
magnitudo=[]
audiodata=[]

def pred(length=1, filename=None, thres=0):
    global cumulated_status, cseq, model,magnitudo, audiodata
    magnitudo=[]
    audiodata=[]

    try:
        import sounddevice as sd

        #samplerate = sd.query_devices(args.device, 'input')['default_samplerate']
        samplerate = 16000.0

        delta_f = (high - low) / (80 - 1)
        fftsize = np.ceil(samplerate / delta_f).astype(int)
        low_bin = int(np.floor(low / delta_f))

        cumulated_status = sd.CallbackFlags()
        cseq = 0
        def callback(indata, frames, time, status):
            global cumulated_status, cseq, magnitudo
            cumulated_status |= status
            
            cseq += 1
            
            if any(indata):
                magnitude = np.abs(np.fft.rfft(indata[:, 0], n=fftsize))
                magnitude *= gain / fftsize
                audiodata.extend(itertools.chain(indata.tolist()))

                magnitudo.append(magnitude)
                
                if cseq == 5:
                    cseq = 0
                    try:
                        y = model.predict(x=np.array(magnitudo).reshape(1,75,334))
                        cls_pred = np.argmax(y,axis=1)
                        #print(".")
                        if y.max()>0.94:
                            ##print('.')
                            print(list(LABEL_TO_INDEX_MAP.keys())[cls_pred[0]])
                    except Exception as e:
                        #print(e)
                        pass
                    magnitudo=[]
                    
        t0=time.time()
        with sd.InputStream(device=None, channels=1, callback=callback,
                            blocksize=int(samplerate * DURATION / 1000),
                            samplerate=samplerate):
            
            while True:
                t1=time.time()
                if t1-t0 > float(length):
                    if filename!=None: soundfile.write(filename,audiodata,16000)
                    return np.array(magnitudo)


        if cumulated_status:
            logging.warning(str(cumulated_status))
    except Exception as e:
        print(e)

In [478]:
import time
import itertools
a = pred(length=5,filename="rekam.wav")

In [381]:
a = np.array([get_spec('record.wav')])

In [382]:
import IPython.display as ipd
ipd.Audio('record.wav')

In [383]:
a.shape

(1, 75, 334)

In [384]:
y_pred = model.predict(x=a)

In [385]:
cls = np.argmax(y_pred)

In [386]:
print(list(LABEL_TO_INDEX_MAP.keys())[list(LABEL_TO_INDEX_MAP.values()).index(cls)])

cipi


In [12]:
list(LABEL_TO_INDEX_MAP.keys())[[1]])

SyntaxError: invalid syntax (<ipython-input-12-b38cfb91488c>, line 1)

In [15]:
from tensorflow.python.keras.models import load_model
model = load_model('myfirst-model.ker')


NameError: name 'LABEL_TO_INDEX_MAP' is not defined